In [1]:
from emrspark import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
conf.set("fs.s3n.awsAccessKeyId","***")
conf.set("fs.s3n.awsSecretAccessKey","***/RUQ")
spark = SparkSession.builder.config(conf=conf).appName('Graph HW3').getOrCreate()

In [2]:
a2q_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ') \
  .load("s3n://upenn-bigdataanalytics/data/sx-stackoverflow-a2q.txt")

In [3]:
graph_sdf = a2q_sdf.selectExpr("cast(_c0 as integer) as from_node", "cast(_c1 as integer) as to_node")

graph_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)



In [4]:
c2q_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ') \
  .load("s3n://upenn-bigdataanalytics/data/sx-stackoverflow-c2q.txt")

In [5]:
c2q_sdf = c2q_sdf.selectExpr("cast(_c0 as integer) as from_node", "cast(_c1 as integer) as to_node")

In [6]:
graph_sdf = graph_sdf.unionAll(c2q_sdf)

In [7]:
c2a_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ') \
  .load("s3n://upenn-bigdataanalytics/data/sx-stackoverflow-c2a.txt")

In [8]:
c2a_sdf = c2a_sdf.selectExpr("cast(_c0 as integer) as from_node", "cast(_c1 as integer) as to_node")

In [9]:
graph_sdf = graph_sdf.unionAll(c2a_sdf)

In [10]:
def transitive_closure(graph_sdf, origins_sdf, depth):
    graph_sdf.repartition(100, "from_node")
    graph_sdf.createOrReplaceTempView("graph_sdf")
    frontier_sdf = origins_sdf
    results_sdf = origins_sdf
    for _ in range(depth):
        frontier_sdf.createOrReplaceTempView("frontier_sdf")
        frontier_sdf = spark.sql("select graph_sdf.to_node as node \
                                  from frontier_sdf inner join graph_sdf \
                                  where graph_sdf.from_node = frontier_sdf.node").cache()
        frontier_sdf = frontier_sdf.subtract(results_sdf)
        if (not sdf_is_empty(frontier_sdf)):
            results_sdf = results_sdf.unionAll(frontier_sdf)
    return results_sdf

def sdf_is_empty(sdf):
    try:
        sdf.take(1)
        return False
    except:
        return True

In [11]:
def transitive_closure(graph_sdf, origins_sdf, depth):
    graph_sdf.repartition(100, "from_node")
    graph_sdf.createOrReplaceTempView("graph_sdf")
    frontier_sdf = origins_sdf
    results_sdf = origins_sdf
    for _ in range(depth):
        frontier_sdf.createOrReplaceTempView("frontier_sdf")
        frontier_sdf = spark.sql("select graph_sdf.to_node as node \
                                  from frontier_sdf inner join graph_sdf \
                                  where graph_sdf.from_node = frontier_sdf.node").cache()
        frontier_sdf = frontier_sdf.subtract(results_sdf)
        if (not sdf_is_empty(frontier_sdf)):
            results_sdf = results_sdf.unionAll(frontier_sdf)
    return results_sdf

def sdf_is_empty(sdf):
    try:
        sdf.take(1)
        return False
    except:
        return True

## Step 2.3

In [12]:
graph_sdf.repartition(100, "from_node")
graph_sdf.createOrReplaceTempView("graph_sdf")
nodes_raw_sdf = spark.sql("select from_node as node1, to_node as node2 from graph_sdf where from_node < 8 or to_node < 8").cache()
nodes_raw_sdf.createOrReplaceTempView('nodes_raw_sdf')
nodes_sdf = spark.sql('select node1 as node from nodes_raw_sdf').union(spark.sql('select node2 as node from nodes_raw_sdf')).cache()
nodes_sdf = nodes_sdf.drop_duplicates(subset=["node"])
nodes_sdf = nodes_sdf[nodes_sdf['node'] < 8]

In [16]:
reachable_sdf = transitive_closure(graph_sdf, nodes_sdf, 3)

In [17]:
reachable_sdf.count()

1905104

In [18]:
reachable_sdf.show()

+------+
|  node|
+------+
|     1|
|     3|
|     5|
|     4|
|     2|
| 17389|
|179115|
|408870|
|  4219|
| 36706|
| 30183|
|   392|
| 42348|
|738811|
|269578|
|  3488|
| 42754|
| 33690|
|113570|
| 17712|
+------+
only showing top 20 rows

